# Validation

OpenFisca-UK runs unit and integration tests on each new version (see [here](https://github.com/PSLmodels/openfisca-uk/tree/master/tests)).
In addition, the table below shows the aggregates produced by the model for the major taxes and benefits, and comparisons with UKMOD (latest [country report](https://www.iser.essex.ac.uk/research/publications/working-papers/cempa/cempa7-20.pdf)) and official sources.[^1]
UKMOD and administrative sources refer to 2018, and OpenFisca-UK is simulated on policy at the end of 2018.
Numbers are in billions of pounds.

[^1]: From the UKMOD country report: unless otherwise specified: Department for Work and Pensions https://www.gov.uk/government/publications/benefit-expenditure-and-caseload-tables-2018 ; Best Start Grant: https://www2.gov.scot/Topics/Statistics/Browse/Social-Welfare/SocialSecurityforScotland/BSGJune2019; Child tax credit and working tax credit: HMRC statistics 
https://www.gov.uk/government/statistics/child-and-working-tax-credits-statistics-finalised-annual-awards-2016-to-2017; Scottish Child Payment: Scottish Fiscal Commission https://www.fiscalcommission.scot/forecast/supplementary-costing-scottish-child-payment; Scottish Child Winter Heating Assistance: Scottish Fiscal Commission 
https://www.fiscalcommission.scot/forecast/supplementary-costing-child-winter-heating-assistance; Income tax: HMRC statistics https://www.gov.uk/government/statistics/income-tax-liabilities-statistics-tax-year-2014-to-2015-to-tax-year-2017-to-2018; National Insurance Contributions: ONS Blue Book Table 5.2.4s 

## Aggregate tables

OpenFisca-UK uprates input FRS data: below are comparisons between the aggregates calculated by OpenFisca-UK, UKMOD and external sources.

### Aggregates in full

In [1]:
import numpy as np
import pandas as pd
from openfisca_uk import Microsimulation

sim = Microsimulation(duplicate_records=2)

_ = np.nan
VARIABLES = [
    "income_tax",
    "total_NI",
    "universal_credit",
    "working_tax_credit",
    "child_tax_credit",
    "child_benefit",
    "housing_benefit",
    "pension_credit",
    "income_support",
    "JSA_income",
    "council_tax_less_benefit",
    "state_pension",
    "ESA_income",
]

df = pd.concat(
    [
        (sim.df(VARIABLES, map_to="household", period=year).sum() / 1e9)
        for year in range(2018, 2023)
    ],
    axis=1,
)
df.columns = list(range(2018, 2023))
df.index = [
    sim.simulation.tax_benefit_system.variables[var].label for var in df.index
]
df
ukmod_df = pd.DataFrame(
    {
        "Income Tax": [163.7, 165.9, 165.0, 173.9, 186.3],
        "National Insurance (total)": [138.6, 144.2, 141.6, 148.0, 152.3],
        "Universal Credit": [11.7, 24.8, 41.3, 40.4, 47.6],
        "Working Tax Credit": [2.5, 1.6, 1.3, 0.6, 0.2],
        "Child Tax Credit": [11.4, 7.1, 4.4, 2.8, 1.2],
        "Housing Benefit": [15.1, 11.0, 8.6, 7.5, 6.1],
        "Child Benefit": [11.5, 11.4, 11.6, 11.6, 11.7],
        "Pension Credit": [4.1, 3.6, 3.6, 2.9, 3.2],
        "ESA (income-based)": [6.4, 5.1, 3.0, 2.3, 0.9],
        "Income Support": [_, _, _, _, _],
        "JSA (income-based)": [_, _, _, _, _],
        "Council Tax (less CTB)": [_, _, _, _, _],
        "State Pension": [_, 88.1, 91.0, 92.6, 95.0],
    }
).T
ukmod_df.columns = list(range(2018, 2023))
# source: https://www.microsimulation.ac.uk/wp-content/uploads/2020/10/cempa7-20.pdf#page=130
# where missing, UKMOD does not separate benefits and therefore figures cannot be obtained

statistics = sim.simulation.tax_benefit_system.parameters.calibration
get_yearly = lambda param, multiplier: np.array(
    [
        round(param(f"{year}-01-01") * multiplier, 1)
        for year in range(2018, 2023)
    ]
)
external_df = pd.DataFrame(
    {
        "Income Tax": get_yearly(statistics.aggregate.income_tax, 1e-9),
        "National Insurance (total)": get_yearly(
            statistics.aggregate.total_NI, 1e-9
        ),
        "Universal Credit": get_yearly(
            statistics.aggregate.universal_credit, 1e-9
        ),
        "Working Tax Credit": get_yearly(
            statistics.aggregate.working_tax_credit, 1e-9
        ),
        "Child Tax Credit": get_yearly(
            statistics.aggregate.child_tax_credit, 1e-9
        ),
        "Housing Benefit": get_yearly(
            statistics.aggregate.housing_benefit, 1e-9
        ),
        "Child Benefit": get_yearly(statistics.aggregate.child_benefit, 1e-9),
        "Pension Credit": get_yearly(
            statistics.aggregate.pension_credit, 1e-9
        ),
        "Income Support": get_yearly(
            statistics.aggregate.income_support, 1e-9
        ),
        "JSA (income-based)": get_yearly(
            statistics.aggregate.JSA_income, 1e-9
        ),
        "Council Tax (less CTB)": get_yearly(
            statistics.aggregate.council_tax_less_benefit, 1e-9
        ),
        "State Pension": get_yearly(statistics.aggregate.state_pension, 1e-9),
        "ESA (income-based)": get_yearly(
            statistics.aggregate.ESA_income, 1e-9
        ),
    }
).T
external_df.columns = list(range(2018, 2023))

df = df.drop(2018, axis=1)
ukmod_df = ukmod_df.drop(2018, axis=1)
external_df = external_df.drop(2018, axis=1)
pd.concat(
    [df.apply(lambda col: col.round(1)), ukmod_df, external_df],
    axis=1,
    keys=["OpenFisca-UK", "UKMOD", "External"],
).fillna("")

[    0.             0.             0.         ...     0.
     0.         11665.08007812]


ValueError: Input [    0.             0.             0.         ...     0.
     0.         11665.08007812] is not a valid value for the entity person (size = 45466 != 86628 = count)

### Differences

#### Absolute

In [ ]:
pd.concat(
    [
        external_df,
        (ukmod_df - external_df).round(1).fillna(""),
        (df - external_df).round(1).fillna(""),
    ],
    axis=1,
    keys=[
        "External",
        "UKMOD Difference (£bn)",
        "OpenFisca-UK Difference (£bn)",
    ],
).fillna("")

#### Relative

In [ ]:
rel_agg_df = pd.concat(
    [
        external_df,
        ((ukmod_df / external_df - 1).round(3) * 100).fillna(""),
        ((df / external_df - 1).round(3) * 100).fillna(""),
    ],
    axis=1,
    keys=["External", "UKMOD Difference (%)", "OpenFisca-UK Difference (%)"],
).fillna("")
rel_agg_df

In [ ]:
shared_columns = [
    col
    for col in df.index
    if col in ukmod_df.index and col in external_df.index
]

df = df.loc[shared_columns]
ukmod_df = ukmod_df.loc[shared_columns]
external_df = external_df.loc[shared_columns]

ukmod_diff = ukmod_df / external_df - 1
openfisca_uk_diff = df / external_df - 1
ukmod_diff = pd.melt(ukmod_diff.reset_index(), id_vars="index")
ukmod_diff.columns = ["Program", "Year", "Relative error"]
openfisca_uk_diff = pd.melt(openfisca_uk_diff.reset_index(), id_vars="index")
openfisca_uk_diff.columns = ["Program", "Year", "Relative error"]


ukmod_agg = pd.melt(ukmod_df.reset_index(), id_vars="index")
ukmod_agg.columns = ["Program", "Year", "Aggregate"]
openfisca_uk_agg = pd.melt(df.reset_index(), id_vars="index")
openfisca_uk_agg.columns = ["Program", "Year", "Aggregate"]
official_agg = pd.melt(external_df.reset_index(), id_vars="index")
official_agg.columns = ["Program", "Year", "Aggregate"]


diff = pd.DataFrame(
    {
        "Program": openfisca_uk_diff.Program,
        "Year": openfisca_uk_diff.Year,
        "OpenFisca-UK": openfisca_uk_diff["Relative error"].abs(),
        "OpenFisca-UK aggregate": openfisca_uk_agg.Aggregate.round(1),
        "UKMOD": ukmod_diff["Relative error"].apply(
            lambda x: abs(x) if x != "" else -1
        ),
        "UKMOD aggregate": ukmod_agg.Aggregate,
        "Official aggregate": official_agg.Aggregate,
    }
).set_index("Program")

diff["Program"] = diff.index

diff = diff[diff.UKMOD >= 0]

diff = diff.sort_values(["Year", "OpenFisca-UK"])

import plotly.express as px

hovertemplate = "<b>%{customdata[4]} in %{customdata[3]}</b><br>Error: %{x}<br>Official: £%{customdata[2]}bn<br>OpenFisca-UK: £%{customdata[0]}bn"

fig = (
    px.bar(
        diff,
        x=["OpenFisca-UK", "UKMOD"],
        orientation="h",
        animation_frame="Year",
        barmode="group",
        color_discrete_map={"OpenFisca-UK": "blue", "UKMOD": "lightgrey"},
        custom_data=[
            "OpenFisca-UK aggregate",
            "UKMOD aggregate",
            "Official aggregate",
            "Year",
            "Program",
        ],
    )
    .update_layout(
        width=800,
        height=600,
        xaxis_tickformat=".0%",
        xaxis_title="Relative error",
        title="Relative aggregate error",
        xaxis_range=(0, 1),
        template="plotly_white",
        legend_title="Model",
    )
    .update_traces(hovertemplate=hovertemplate)
)

for frame in fig.frames:
    for data in frame.data:
        data.hovertemplate = hovertemplate

fig

## Caseload tables

OpenFisca-UK uprates input FRS data: below are comparisons between the aggregates calculated by OpenFisca-UK, UKMOD and external sources.

### Caseloads in full

In [ ]:
import numpy as np
import pandas as pd
from openfisca_uk import Microsimulation

sim = Microsimulation(duplicate_records=2)

_ = np.nan
VARIABLES = [
    "income_tax",
    "universal_credit",
    "working_tax_credit",
    "child_tax_credit",
    "child_benefit",
    "housing_benefit",
    "pension_credit",
    "income_support",
    "JSA_income",
    "state_pension",
    "ESA_income",
]


def get_caseload(variable, year):
    entity = sim.simulation.tax_benefit_system.variables[variable].entity.key
    value = sim.calc(variable, period=year).values > 0
    household_level = sim.map_to(value, entity, "household")
    return (
        sim.calc("household_weight", period=year).values * household_level
    ).sum() / 1e6


df = pd.concat(
    [
        (
            pd.Series(
                {
                    variable: get_caseload(variable, year)
                    for variable in VARIABLES
                }
            )
        )
        for year in range(2018, 2023)
    ],
    axis=1,
)
df.columns = list(range(2018, 2023))
df.index = [
    sim.simulation.tax_benefit_system.variables[var].label for var in df.index
]
df
ukmod_df = pd.DataFrame(
    {
        "Income Tax": [_, 29.3, 29.4, 29.9, 30.0],
        "Universal Credit": [_, 3.0, 4.6, 4.8, 5.6],
        "Working Tax Credit": [_, 0.5, 0.4, 0.2, 0.1],
        "Child Tax Credit": [_, 1.5, 0.9, 0.6, 0.2],
        "Housing Benefit": [_, 2.6, 2.0, 1.8, 1.5],
        "Child Benefit": [_, 7.2, 7.2, 7.1, 7.1],
        "Pension Credit": [_, 1.5, 1.5, 1.3, 1.3],
        "Income Support": [_, _, _, _, _],
        "JSA (income-based)": [_, _, _, _, _],
        "ESA (income-based)": [_, 0.8, 0.5, 0.3, 0.1],
    }
).T
ukmod_df.columns = list(range(2018, 2023))
# source: https://www.microsimulation.ac.uk/wp-content/uploads/2020/10/cempa7-20.pdf#page=130
# where missing, UKMOD does not separate benefits and therefore figures cannot be obtained

statistics = sim.simulation.tax_benefit_system.parameters.calibration
get_yearly = lambda param, multiplier: [
    round(param(f"{year}-01-01") * multiplier, 1) for year in range(2018, 2023)
]
external_df = pd.DataFrame(
    {
        "Income Tax": get_yearly(statistics.count.income_tax, 1e-6),
        "Universal Credit": get_yearly(
            statistics.count.universal_credit, 1e-6
        ),
        "Working Tax Credit": get_yearly(
            statistics.count.working_tax_credit, 1e-6
        ),
        "Child Tax Credit": get_yearly(
            statistics.count.child_tax_credit, 1e-6
        ),
        "Housing Benefit": get_yearly(statistics.count.housing_benefit, 1e-6),
        "Child Benefit": get_yearly(statistics.count.child_benefit, 1e-6),
        "Pension Credit": get_yearly(statistics.count.pension_credit, 1e-6),
        "Income Support": get_yearly(statistics.count.income_support, 1e-6),
        "JSA (income-based)": get_yearly(statistics.count.JSA_income, 1e-6),
        "State Pension": get_yearly(statistics.count.state_pension, 1e-6),
        "ESA (income-based)": get_yearly(statistics.count.ESA_income, 1e-6),
    }
).T
external_df.columns = list(range(2018, 2023))

df = df.drop(2018, axis=1)
ukmod_df = ukmod_df.drop(2018, axis=1)
external_df = external_df.drop(2018, axis=1)
pd.concat(
    [df.apply(lambda col: col.round(1)), ukmod_df, external_df],
    axis=1,
    keys=["OpenFisca-UK", "UKMOD", "External"],
).fillna("")

### Differences

#### Absolute

In [ ]:
pd.concat(
    [
        external_df,
        (ukmod_df - external_df).round(1).fillna(""),
        (df - external_df).round(1).fillna(""),
    ],
    axis=1,
    keys=[
        "External",
        "UKMOD Difference (m)",
        "OpenFisca-UK Difference (m)",
    ],
).fillna("")

#### Relative

In [ ]:
pd.concat(
    [
        external_df,
        ((ukmod_df / external_df - 1).round(3) * 100).fillna(""),
        ((df / external_df - 1).round(3) * 100).fillna(""),
    ],
    axis=1,
    keys=["External", "UKMOD Difference (%)", "OpenFisca-UK Difference (%)"],
).fillna("")

In [ ]:
shared_columns = [
    col
    for col in df.index
    if col in ukmod_df.index and col in external_df.index
]

df = df.loc[shared_columns]
ukmod_df = ukmod_df.loc[shared_columns]
external_df = external_df.loc[shared_columns]

ukmod_diff = ukmod_df / external_df - 1
openfisca_uk_diff = df / external_df - 1
ukmod_diff = pd.melt(ukmod_diff.reset_index(), id_vars="index")
ukmod_diff.columns = ["Program", "Year", "Relative error"]
openfisca_uk_diff = pd.melt(openfisca_uk_diff.reset_index(), id_vars="index")
openfisca_uk_diff.columns = ["Program", "Year", "Relative error"]


ukmod_agg = pd.melt(ukmod_df.reset_index(), id_vars="index")
ukmod_agg.columns = ["Program", "Year", "Aggregate"]
openfisca_uk_agg = pd.melt(df.reset_index(), id_vars="index")
openfisca_uk_agg.columns = ["Program", "Year", "Aggregate"]
official_agg = pd.melt(external_df.reset_index(), id_vars="index")
official_agg.columns = ["Program", "Year", "Aggregate"]


diff = pd.DataFrame(
    {
        "Program": openfisca_uk_diff.Program,
        "Year": openfisca_uk_diff.Year,
        "OpenFisca-UK": openfisca_uk_diff["Relative error"].abs(),
        "OpenFisca-UK aggregate": openfisca_uk_agg.Aggregate.round(1),
        "UKMOD": ukmod_diff["Relative error"].apply(
            lambda x: abs(x) if x != "" else -1
        ),
        "UKMOD aggregate": ukmod_agg.Aggregate,
        "Official aggregate": official_agg.Aggregate,
    }
).set_index("Program")

diff["Program"] = diff.index

diff = diff[diff.UKMOD >= 0]

diff = diff.sort_values(["Year", "OpenFisca-UK"])

import plotly.express as px

hovertemplate = "<b>%{customdata[4]} in %{customdata[3]}</b><br>Error: %{x}<br>Official: £%{customdata[2]}bn<br>OpenFisca-UK: £%{customdata[0]}bn"

fig = (
    px.bar(
        diff,
        x=["OpenFisca-UK", "UKMOD"],
        orientation="h",
        animation_frame="Year",
        barmode="group",
        color_discrete_map={"OpenFisca-UK": "blue", "UKMOD": "lightgrey"},
        custom_data=[
            "OpenFisca-UK aggregate",
            "UKMOD aggregate",
            "Official aggregate",
            "Year",
            "Program",
        ],
    )
    .update_layout(
        width=800,
        height=600,
        xaxis_tickformat=".0%",
        xaxis_title="Relative error",
        title="Relative caseload error",
        xaxis_range=(0, 1),
        template="plotly_white",
        legend_title="Model",
    )
    .update_traces(hovertemplate=hovertemplate)
)

for frame in fig.frames:
    for data in frame.data:
        data.hovertemplate = hovertemplate

fig

## Automated tests

Below are test results from the most recent version.

In [ ]:
from openfisca_uk.tests.microsimulation.test_statistics import tests

pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_rows", 500)
pd.DataFrame({"Name": tests, "Passed": [test.test()[0] for test in tests]})